# Import Libraries

In [63]:
import pandas as pd
import numpy as np

import re

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier

# Data Analysis

In [64]:
from google.colab import drive
drive.mount('/content/drive')

fake = pd.read_csv("/content/drive/MyDrive/AICTE/fake.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [66]:
genuine = pd.read_csv("/content/drive/MyDrive/AICTE/true.csv")

In [67]:
genuine.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [68]:
fake['label']=1
genuine['label']=0

In [69]:
fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [70]:
genuine.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [71]:
fake.shape

(23481, 5)

In [72]:
genuine.shape


(21417, 5)

In [73]:
articles_data = pd.concat([fake,genuine],ignore_index=True)

In [74]:
articles_data.shape

(44898, 5)

In [75]:
articles_data.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [76]:
articles_data.tail()

,title,text,subject,date,label
44893,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",0
44894,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",0
44895,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",0
44896,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",0
44897,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",0


In [77]:
articles_data.drop(columns=['subject','date','text'],inplace=True)

In [78]:
articles_data.head()

,title,label
0,Donald Trump Sends Out Embarrassing New Year’...,1
1,Drunk Bragging Trump Staffer Started Russian ...,1
2,Sheriff David Clarke Becomes An Internet Joke...,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,1
4,Pope Francis Just Called Out Donald Trump Dur...,1


# Data Pre Processing

In [79]:
word_lem=WordNetLemmatizer()

In [80]:
import nltk

# Download the punkt_tab resource
nltk.download('punkt_tab')

# Download the stopwords resource as well, as it is used in the clean_data function
nltk.download('stopwords')

# Download the wordnet resource for lemmatization
nltk.download('wordnet')

def clean_data(text):
  words = []
  word_tokens = word_tokenize(text)
  for word in word_tokens:
    # Make sure the word is not empty after cleaning
    cleaned_word = re.sub('[^a-zA-Z]','',word).lower()
    if cleaned_word and cleaned_word not in stopwords.words('english'):
      result=word_lem.lemmatize(cleaned_word)
      words.append(result)
  return ' '.join(words)

articles_data['title']=articles_data['title'].apply(clean_data)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Cleaning Data

In [81]:
input=articles_data['title']
output=articles_data['label']

In [82]:
x_train,x_test,y_train,y_test=train_test_split(input,output,test_size=0.2)

In [83]:
print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(35918,) (8980,) (35918,) (8980,)


In [84]:
vector=TfidfVectorizer()

In [85]:
x_train_vector = vector.fit_transform(x_train)
x_test_vector = vector.transform(x_test)

# Model Creation

In [86]:
model=PassiveAggressiveClassifier(max_iter=50,random_state=7)

In [87]:
model.fit(x_train_vector, y_train)

PassiveAggressiveClassifier(max_iter=50, random_state=7)

# Model Evaluation

In [88]:
pred = model.predict(x_test_vector)

In [89]:
pred

array([0, 0, 0, ..., 0, 1, 0])

In [90]:
from sklearn.metrics import accuracy_score

In [91]:
accuracy_score(pred, y_test)

0.9368596881959911

In [113]:
def predict(model_name, text):
  input_dataset = pd.DataFrame([text], columns=['title'])
  input_dataset['title'] = input_dataset['title'].apply(clean_data)
  text_vector = vector.transform(input_dataset['title'])
  result = model_name.predict(text_vector)

  if result[0]==0:
    return 'This is a Genuine News'
  if result[0]==1:
    return 'This is a Fake News'

In [114]:
predict(model, ' BREAKING: Cop Finally Gets His Due, Walter Scottâ€™s Killer Sentenced To Prison (DETAILS)')

'This is a Fake News'

In [115]:
predict(model, 'U.S. military to accept transgender recruits on Monday: Pentagon')

'This is a Genuine News'

In [116]:
predict(model, 'Pope Francis Just Called Out Donald Trump During His Christmas Speech')

'This is a Fake News'